In [1]:
# importing pandas as pd
import pandas as pd
import math
import matplotlib.pyplot as plt
import seaborn as sns
import os
import json

In [2]:
device_dir_paths = [
    'AwairAirQuality',
    'LiFXBulb',
    'PixStarPhotoFrame',
    'RingDoorBell',
    'TPLinkCamera',
    'TribySpeaker',

    # 'AmazonEcho/1',
    # 'AmazonEcho/2',
    # 'AmazonEcho/3',
    # 'AmazonEcho/4',
    # 'AmazonEcho/5',
    # 'AmazonEcho/6',
    # 'AmazonEcho/7',
    # 'AmazonEcho/8',
    # 'AmazonEcho/9',
    'AmazonEcho',

    # 'NestProtect/1',
    # 'NestProtect/2',
    # 'NestProtect/3',
    # 'NestProtect/4',
    # 'NestProtect/5',
    # 'NestProtect/6',
    # 'NestProtect/7',

    # 'WithingsSleepSensor/1',
    # 'WithingsSleepSensor/2',
    # 'WithingsSleepSensor/3',
    # 'WithingsSleepSensor/4',
    'WithingsSleepSensor',

    'SamsungCamera',
    'WithingsBabyMonitor'
]

In [3]:
root_dir_path = '../../data/bestPracticeViolations'
output_file = '../../results/bestPracticeViolations/ssdp-violations.csv' 

In [4]:
'''
data_device =    
    {
        device: {
            protocol: df_of_violations.csv
            ...
        } ,
        ...
    }
'''

# Create an empty list to store the data from CSV files
data_device = {}

for device_dir_path in device_dir_paths:
    # this is needed because for outputs from the java script the separator is ',,,'.
    # but for devices with concatanated attribute files the separator is '/'
    if device_dir_path == 'SamsungCamera' or device_dir_path == 'AmazonEcho' or device_dir_path == 'WithingsSleepSensor':
        sepa = '|'
    else:
        sepa = ',,,'
    data_proto = {}
    directory_path  = os.path.join(root_dir_path, device_dir_path)
    # Get a list of all files in the directory
    file_list = os.listdir(directory_path)
    print(file_list)

    # Iterate over each file in the directory
    for file_name in file_list:
        print(f'file_name: {file_name}')
        # reading only the violations files
        if 'violations' in file_name:
            proto = file_name.split('-')[0]
            # Check if the file is a CSV file
            if file_name.endswith('.csv'):
                # Create the file path
                file_path = os.path.join(directory_path, file_name)
                
                # Read the CSV file and append its data to the list
                df = pd.read_csv(file_path, sep=',')
                # data.append(df)

                data_proto[proto] = df
                # data_proto[proto] = 'df'
    
    data_device[device_dir_path] = data_proto

# # Write dictionary as JSON
# with open('output.json', 'w') as json_file:
#     json.dump(data_device, json_file)

# data_device['AwairAirQuality']['tls'][' violations']







['tlsattributes.csv', 'tls-violations.csv', 'ntp-violations.csv', 'dnsattributes.csv', 'dns-violations.csv', 'dhcpattributes.csv', 'icmp-violations.csv', 'icmpattributes.csv', 'dhcp-violations.csv', 'ntpattributes.csv']
file_name: tlsattributes.csv
file_name: tls-violations.csv
file_name: ntp-violations.csv
file_name: dnsattributes.csv
file_name: dns-violations.csv
file_name: dhcpattributes.csv
file_name: icmp-violations.csv
file_name: icmpattributes.csv
file_name: dhcp-violations.csv
file_name: ntpattributes.csv
['tlsattributes.csv', 'tls-violations.csv', 'ntp-violations.csv', 'dnsattributes.csv', 'dns-violations.csv', 'isakmpattributes.csv', 'dhcpattributes.csv', 'icmp-violations.csv', 'icmpattributes.csv', 'isakmp-violations.csv', 'dhcp-violations.csv', 'ntpattributes.csv']
file_name: tlsattributes.csv
file_name: tls-violations.csv
file_name: ntp-violations.csv
file_name: dnsattributes.csv
file_name: dns-violations.csv
file_name: isakmpattributes.csv
file_name: dhcpattributes.csv
fi

In [5]:
'''
unique_viol_device =    
    {
        device: {
            protocol: {
                violation: number_of_ccurences,
                ...
                'unique_flows_with_violations': [list of unique flows]
                'num_violated_flows': len(unique_flows_with_violations)
            },
            ...
        } ,
        ...
    }
'''

# srcMac, dstMac, ethType, srcIp, dstIp, ipProto, srcPort, dstPort

unique_viol_device = {}
for device, protos in data_device.items():
    print('device:', device)
    unique_viol_proto = {}
    for proto, df in protos.items():
        if proto == 'ssdp':  # processing for only a selected protocol
            print(f'\t{device} --> proto: {proto}')
            viol = {}
            num_violated_flows = 0
            unique_flows_with_violations = []

            for index in df.index:
                print(f'df.columns: {df.columns}')
                value_viol = df[' violations'][index]
                # making a flow signature
                value_flow = df[' srcIp'][index] + '-' + df[' dstIp'][index] + '-' + str(df[' ipProto'][index]) + '-' + str(df[' srcPort'][index]) + '-' + str(df[' dstPort'][index])
                
                if value_viol not in viol:
                    viol[value_viol] = 0
                viol[value_viol] += 1

                if value_flow not in unique_flows_with_violations:  # adding unique flow with a violation
                    unique_flows_with_violations.append(value_flow)  
                    num_violated_flows += 1

            viol['unique_flows_with_violations'] = []  # unique_flows_with_violations  # commented out because not needed at this point, can uncomment to be used later
            viol['num_violated_flows'] = num_violated_flows
            unique_viol_proto[proto] = viol

    unique_viol_device[device] = unique_viol_proto

# with open('output2.json', 'w') as json_file:
#     json.dump(unique_viol_device, json_file)


device: AwairAirQuality
device: LiFXBulb
device: PixStarPhotoFrame
	PixStarPhotoFrame --> proto: ssdp
device: RingDoorBell
device: TPLinkCamera
device: TribySpeaker
device: AmazonEcho
	AmazonEcho --> proto: ssdp
device: WithingsSleepSensor
	WithingsSleepSensor --> proto: ssdp
device: SamsungCamera
	SamsungCamera --> proto: ssdp
device: WithingsBabyMonitor


In [6]:
df_to_save = pd.DataFrame(unique_viol_device)

# Specify the output CSV file path
# output_file = '../../results/bestPracticeViolations/v3.csv'  

# Write the DataFrame to the CSV file
df_to_save.to_csv(output_file, index=True, sep='|')

df_to_save.head()

,AwairAirQuality,LiFXBulb,PixStarPhotoFrame,RingDoorBell,TPLinkCamera,TribySpeaker,AmazonEcho,WithingsSleepSensor,SamsungCamera,WithingsBabyMonitor
ssdp,NaN,NaN,"{'unique_flows_with_violations': [], 'num_viol...",NaN,NaN,NaN,"{'unique_flows_with_violations': [], 'num_viol...","{'unique_flows_with_violations': [], 'num_viol...","{'unique_flows_with_violations': [], 'num_viol...",NaN
